## Prerequisite

After compiling the DFT driver and installing PyCDFT, run the ground state calculation.
- - - - - - -
For Qbox

```bash
 export qb="/path/to/executable"
 $qb < gs.in > gs.out
```
Then in the same directory, run [Qbox in server mode](qboxcode.org/daoc/html/usage/client-server.html) (using interactive queue)
    

```bash
 mpirun -np <ntasks> $qb -server qb_cdft.in qb_cdft.out
```

where qb_cdft.\* are files reserved in client-server mode.

### Tutorial: coupling constant for He$_2^+$

In [1]:
from pycdft import *
from ase.io import read

In [2]:
cell = read("./He2.cif")
print(r"Initial atomic positions (Ang):")
print(cell.get_positions())
print(cell.get_cell())

Initial atomic positions (Ang):
[[ 0.  0.  0.]
 [ 0.  0.  3.]]
[[ 15.87531   0.        0.     ]
 [  0.       15.87531   0.     ]
 [  0.        0.       15.87531]]


In [3]:
d = 3.0
#V = (-1,1)
V = -0.704679584617

print("==================== Initializing Run =========================")
# load sample geometry
cell.positions[1][2] = d 
sample = Sample(ase_cell=cell, n1=140, n2=140, n3=140, vspin=1)
print(sample.atoms[1])
    
# load DFT driver
qboxdriver = QboxDriver(
    sample=sample,
    init_cmd="load gs.xml \n" 
        "set xc PBE \n" 
        "set wf_dyn PSDA \n" 
        "set_atoms_dyn CG \n" 
        "set scf_tol 1.0E-8 \n",
        scf_cmd="run 0 50 5",
)
    
# set up CDFT constraints and solver
solver1 = CDFTSolver(job="scf", optimizer="secant",sample=sample, dft_driver=qboxdriver)
solver2 = solver1.copy()
    
# add constraint to two solvers
ChargeTransferConstraint(
    sample=solver1.sample,
    donor=Fragment(solver1.sample, solver1.sample.atoms[0:1]),
    acceptor=Fragment(solver1.sample, solver1.sample.atoms[1:2]),
    #V_brak=V,
    V_init = V,
    N0=1,
    N_tol=1E-6
)
ChargeTransferConstraint(
    sample=solver2.sample, 
    donor=Fragment(solver2.sample, solver2.sample.atoms[0:1]),
    acceptor=Fragment(solver2.sample, solver2.sample.atoms[1:2]),
    #V_brak=V,
    V_init = -V,
    N0=-1, 
    N_tol=1E-6
)


print("~~~~~~~~~~~~~~~~~~~~ Applying CDFT ~~~~~~~~~~~~~~~~~~~~")
print("---- solver A ------")
solver1.solve()
print("---- solver B ------")
solver2.solve()
    
print("~~~~~~~~~~~~~~~~~~~~ Calculating coupling ~~~~~~~~~~~~~~~~~~~~")
#compute_elcoupling_symm_nospin(solver1,solver2)
compute_elcoupling(solver1, solver2)
    
print("=========== "+ str(d)+" Bohr"+" =======================")
print("==================== JOB DONE =========================")

==================== Initializing Run =========================
Atom He. Abs coordi (Bohr) (0.000, 0.000, 5.669). 
QboxDriver: setting output path to ./pycdft_outputs/solver1/...
QboxDriver: waiting for Qbox to start...
QboxDriver: initializing Qbox...
Constraint charge transfer: N_tol 0.00010, eps 1.00E-06
Constraint charge transfer: N_tol 0.00010, eps 1.00E-06
~~~~~~~~~~~~~~~~~~~~ Applying CDFT ~~~~~~~~~~~~~~~~~~~~
---- solver A ------
===================== Initializing Run ====================
Updating constraint with new structure...
Running optimizer:  secant
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
SCF iteration 1
  Ed (DFT energy) = -4.733620
  Ec (constraint energy) = -0.704681
  E (Ed + Ec) = -5.438301
  W (free energy) = -4.733621
  > Constraint #0 (type = charge transfer, N0 = 1, V = -0.704679584617):
    N = 1.000000
    dW/dV = N - N0 = -0.00000002
Elapsed time: 
00h:04m:47.82s
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
00h:05m:45.52s
*Constrained SCF converged!*

---- solver B ------
=====